# STEM for Creatives Week 6  - Images

### Pixels

As we saw with audio, when we break media down on computers, they are just multi-dimensional arrays. Whilst audio is often 1-dimensional (or more for multi-channel audio), images have more components to them. 

Whilst digital audio is made up of **samples**, digital images are made up of **pixels**. When we are dealing with black and white images (often known as **grayscale**), each image is a **2D array**, which each dimension relating to 

- row (height)
- column (width)

Each item in this array represents a pixel and its number tells us where on the scale of black (low) to white (high) it is. We can use different types of numbers to represent ecah pixel but often the scale is 0 - 255.

### PIL (Python Imaging Library)

We can use PIL to import and display images, and then turn them into **NumPy** arrays. And we know how to do things with them!

In [ ]:
!pip install pillow scikit-image

In [ ]:
from PIL import Image
import numpy as np

#This is actually a colour image, so we make it grayscale to begin with (convert('L'))
im = np.array(Image.open('images/robot-enstein.jpg').convert('L'))
Image.fromarray(im)

In [ ]:
#How big is it?
h = im.shape[0]
w = im.shape[1]
print(w, h)

In [ ]:
#Looking at pixels
#Gives us the grayscale of a particular pixel 
print(im[12,45])
#Pixel in the middle
mid_y = int(h/2)
mid_x = int(w/2)
print(im[mid_y,mid_x])

### Colour (RGB)

Sure grayscale is good, but have you tried colour? In the RGB representation an image is made up of three channels

- **R**ed
- **G**reen 
- **B**lue 

So in a way its actually like 3 images, that combine together to make the full colour output. We can get all colours from combinations of these three base colours.

### Colour Channels

So how does effect our NumPy array? We end up with a **3D array**, whose dimensions map to 

- row (height)
- column (width)
- color (channel)


In [ ]:
im = np.array(Image.open('images/robot-enstein.jpg'))
Image.fromarray(im)

In [ ]:
#Get the size and number of channels
h = im.shape[0]
w = im.shape[1]
c = im.shape[2]
print(w, h, c)

In [ ]:
#Looking at pixels
#Gives us the RGB of a particular pixel -> We get three values for each one!
print(im[12,45])
#Pixel in the middle
print(im[mid_y,mid_x-60])

### Setting Pixel Values

As well as looking at pixel values, we can also set them to new things. We can do this by just changing the values in the arrays, just like we did with audio files. 

When we change the file, like we did with the audio, we first make a copy. This way the original image file stays unaltered so we can use it again  

```
new_image = im.copy()
```

#### Tuples

We can do this to single pixels, or groups of pixels. Each pixel is actually represented as a **tuple**. Tuples are single objects that hold mutiple values and are a **collection which is ordered and unchangeable**. This means we can access the items inside with indexes (because its ordered), but we cant change them (because they're unchangeable).

Tuples are written, and created, with **round brackets**

```
a = (1,2,3,4)
```

In [ ]:
#Set the middle pixel to full red (RGB). Look reeeeal closely.
new_image = im.copy()
new_image[mid_y,mid_x] = (255,0,0)
Image.fromarray(new_image)

## Making a Photo Mosaic 

Next we will build up some code you to create a photomosaic. 

A chosen image will get reconstructed using a seperate dataset of images that you specify. These images are used for the tiles in the photomosaic, and they are selected based on image with the closest mean colour to the target pixel. 

The code here is a modified version of code from this repository: https://github.com/MstrFunkBass/facemo

In [ ]:
import IPython
import random
from mosaic import get_images

### Step 1 Downsampling

First we need to downsample the source image so it has less pixels. As our plan is swap each source pixel for a matching image from the dataset, it is in our interest to have less of them! 

To do this, we will just skip pixels at regular intervals. This does lose some information, but is really efficient and turns out to work fine for our purposes. 

#### Skip indexing 

``array(start,end,step)``

Here we use the code ``source[::skip,::skip]`` to say "Get me all of the pixels, skipping at a given interval.

Play with the `skip` variable to see the effects of different down sampling. The display in the notebook is not quite accurate however. As we downsample, we are making smaller images (less pixels). To display we have sclaed back up to the original size so it looks more like a **blur** than a **downsample**. If you look at the actual file `scaled.png`, you will see a much more blocky image.


In [ ]:

#Drop the 4th alpha channel we get the a .png
source = np.array(Image.open('images/cat_meme.png'))[:,:,:3]
skip = 20
mosaic_template = source[::skip,::skip]
Image.fromarray(mosaic_template).save('scaled.png')
print(mosaic_template.shape)
IPython.display.Image('scaled.png',width=source.shape[1]//2,height=source.shape[0]//2)


### Step 2 Getting a dataset

We have some code which walks through a file and finds and the images, scales them all to the same size (and smaller) and stores them in a dataset. If you want to see how this works, look in the file `mosaic.py`

Here is the code for loading the dataset of images. First you will need to [download this file](https://artslondon-my.sharepoint.com/:u:/g/personal/t_broad_arts_ac_uk/EaQZzKGUebZMkn_nAHX1VLUBKS4tYSuOqUOH_0BVUSjanA?e=u9W8PI), unzip it, then put it into the `data` folder.

This is a dataset of [images of animals from kaggle](https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals) that have been resized into thumbnails (for faster download and processing). The full dataset should contain 5400 images.

In [ ]:
thumbnail_size = (50,50)
dataset = get_images("data/animal_thumbnails/land_mammals")

In [ ]:
#images x width x height x channels
print(dataset.shape)
Image.fromarray(dataset[random.randint(0,len(dataset))])

### Step 3 Find Average Colour Of Each Image

To help us find which image from the dataset should replace each pixel in the source, we are going to find out what the **average red, green and blue** values are for each image. We need to do this for each channel to maintain the colour information. We will then use this to match to the **red, green and blue** values of each pixel in the downsampled source image.

#### `np.apply_over_axes()`

We we will us a ``numpy`` function called ``apply_over_axes()`` to apply our `np.mean` function to only certain channels of our 4d array.

If we did `np.mean()` to the whole dataset, we'd just get one value! 

We can get the average of all the images in our dataset by applying it to the `first axis [0]`

In [ ]:
image_values = np.apply_over_axes(np.mean, dataset, [0]).astype(np.uint8)
Image.fromarray(image_values[0]).save('mean_cat.png')
IPython.display.Image('mean_cat.png',width=400,height=400)

But what we actually want is to apply it to the second and third axes. This averages the pixel information over the `width` and `height`, but maintains the separation of `images` and `channels`. 

We end up with an array that is `images x 1 x 1 x channels`. We can use `np.reshape` to remove those two in the middle.

In [ ]:
mean_rgb_dataset = np.apply_over_axes(np.mean, dataset, [1,2])
print(mean_rgb_dataset.shape)
mean_rgb_dataset = mean_rgb_dataset.reshape(len(dataset), 3)
print(mean_rgb_dataset.shape)


### Step 4 Match Images to Pixels

Next we have to do the matching! We will use `binary tree search` to do this efficiently, no need to know how this works in detail. 

We build a `tree` from the `mean_rgb_dataset`, and then `query()` with each pixel from the source to get the `k` closest matches. We then randomly pick one and save the index of the matched image for building the mosaic in the next step.

#### Nested `for loops`

Last week we saw how to use `for loops` to iterate over a `1D list` (e.g. an audio file). This enabled us to write one piece of code that got applied to each window of audio in turn.

Images can be seen as `2D lists`, so we can actually use 2 `for loops` to write code to address each pixel in turn! One `for loop` is `nested` inside the other. 

The code below first starts on the top row, then the second `for loop` loops over each column. When it has done each column in the first row, the second loop is over and the first loop then moves onto the second row. We then move through each column again, until we have done all the rows.

```
for row in range(w):
    for col in range(h):
        pixel = mosaic_template[row, col]
```



In [ ]:
from scipy import spatial
import random

In [ ]:
#Build the search tree
tree = spatial.KDTree(mean_rgb_dataset)

#Variables to store which image is assigned to which pixel
mosaic_template = np.swapaxes(mosaic_template,0,1)
w,h = mosaic_template.shape[0:2]
matched_images = np.zeros((w,h), dtype=np.uint32)
#Go through each pixel and find the closest matching thumbnail image by mean colour and assign the index into the 2D array
k = 40
for row in range(w):
    for col in range(h):
        pixel = mosaic_template[row, col]
        #Get the match
        match = tree.query(pixel, k=k)
        pick = random.randint(0, k-1)
        matched_images[row, col] = match[1][pick]

### Step 5 Build the Mosaic

Now we need to stitch all those matched images together into our mosaic. Again a `nested for loop` is useful to work through our `2D data` (again, the pixels of the source image). 

For each pixel, we retrieve the index of the match from `matched_images`. We then get the original thumbnail and `paste()` it into the right grid square on our `mosaic`. 

In [ ]:
#Variable that can contail all the pixel values for the new image
mosaic = Image.new('RGB', (thumbnail_size[0]*w, thumbnail_size[1]*h))

#Go through each pixel in the array of thumbnail<>pixel indexes and then assign all the pixels of the thumbnail into the final array
for i in range(w):
    for j in range(h):
        matched_image = dataset[matched_images[i, j],:,:,:]
        #Coordinates to place the thumbnail
        x, y = i*thumbnail_size[0], j*thumbnail_size[1]
        im = Image.fromarray(matched_image)
        mosaic.paste(im, (x,y))

In [ ]:
#Save the photomosaic to a file
mosaic.save('mosaic.png')

#Display the photomosaic in the notebook
display(mosaic)